# 岭回归和Lasso回归的背景：
岭回归(ridge regression, Tikhonov regularization)是一种专用于共线性数据分析的有偏估计回归方法，实质上是一种改良的最小二乘估计法，通过放弃最小二乘法的无偏性，以损失部分信息、降低精度为代价获得回归系数更为符合实际、更可靠的回归方法，对病态数据的拟合要强于最小二乘法。Lasso回归的全称是Least Absolute Selection and Shrinkage Operator，即最小绝对值选择与收缩算子。Lasso方法是以缩小变量集（降阶）为思想的压缩估计方法。它通过构造一个惩罚函数，可以将变量的系数进行压缩并使某些回归系数变为0，进而达到变量选择的目的。两种回归都可以用来解决标准线性回归的过拟合问题。

# 岭回归的原理：
## 1.    多元回归和岭回归的数学表示
多元线性回归：
$$
f(w)=\sum_{i=1}^{m}\left(y_{i}-x_{i}^{T} w\right)^{2}
$$
岭回归：

$$
f(w)=\sum_{i=1}^{m}\left(y_{i}-x_{i}^{T} w\right)^{2}+\lambda \sum_{i=1}^{n} w_{i}^{2}
$$
从数学表示上看我们可以看出明显的相似性，岭回归的数学表示仅仅是在多元线性回归的基础上增加了一个惩罚项。惩罚项中 λ 的选择一定程度影响了回归的精确性，那么如何选择合适的 λ 呢？

### a. 岭迹分析
要了解岭迹分析，首先得知道岭迹到底是什么。岭迹是指将 λ 从0增加到正无穷的过程得到的中每个 w 分量的变化曲线。
例如：
![Image Name](https://cdn.kesci.com/upload/image/qvx01c7pvm.png?imageView2/0/w/960/h/960)
岭迹法选择 λ 值的一般原则是：
（1）     各回归系数的岭估计基本稳定
（2）     用最小二乘估计时符号不合理的回归系数，其岭估计的符号变得合理
（3）     回归系数没有不合乎经济意义的绝对值
（4）     残差平方和增大不多

### b. VIF(方差膨胀因子)法
当VIF>0时，存在多重共线性，因此我们可以不断增加 λ 直到所有的w的VIF<0。
关于VIF的具体解释，这里就不详细展开，如果有兴趣可以阅读【参考资料3】了解具体原理及其python实现。

### c.    最小化均方预测误差
该方法在实际中用的最多，我们使用K折交叉验证的方法来选择最佳的调整参数。所谓的K折交叉验证,是说将样本数据随机分为K个等分。将第1个子样本作为“验证集” (validation set)而保留不用,而使用其余K-1个子样本作为“训练集” (training set)来估计此模型,再以此预测第1个子样本,并计算第1个子样本的“均方预测误差” (Mean Squared Prediction Error) 。其次,将第2个子样本作为验证集,而使用其余K-1个子样本作为训练集来预测第2个子样本,并计算第2个子样本的MSPE。以此类推,将所有子样本的MSPE加总,即可得整个样本的MSPE,最后,选择调整参数,使得整个样本的MSPE最小,故具有最佳的预测能力。
**同时值得注意的是我们需要保证 λ 的量纲一致，如果出现量纲不一样的情况则可使用标准化的方法。**

#  Lasso回归的原理
 Lasso和岭回归从公式上看和很大的相似性，Lasso回归仅仅将岭回归公式中的惩罚项改为
$$
\sum_{i=1}^{n}\left|w_{i}\right|
$$
与岭回归的不同在于，此约束条件使用了绝对值的一阶惩罚函数代替了平方和的二阶函数。虽然只是形式稍有不同，但是得到的结果却又很大差别。在LASSO中，当λ很小的时候，一些系数会随着变为0而岭回归却很难使得某个系数恰好缩减为0. 我们可以通过几何解释看到LASSO与岭回归之间的不同。Lasso回归也存在一些缺点，Lasso回归无法得出显式解，智能使用近似化的计算法（坐标轴下降法和最小角回归法）估计出来的结果不太稳定，存在一定的误差。


# 岭回归和Lasso回归的代码实现
在sklearn中提供了Lasso回归和岭回归的方法，在本示例中我们将运用这两种算法预测sklearn中提供的波士顿房价的数据集。

## 1. 导入相关模块

In [1]:
# 导入波士顿房价的数据集
from sklearn.datasets import load_boston
#导入分割，标准化,均方误差和R^2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score


## 2.对数据集进行处理

In [2]:
Boston_Price = load_boston()
# 对数据集分割为训练集和测试集
x_train,x_test,y_train,y_test = train_test_split(Boston_Price.data, Boston_Price.target, test_size = 0.2)
# 对数据进行标准化处理处理，消除量纲对结果的影响
std_x = StandardScaler()
x_train = std_x.fit_transform(x_train)
x_test = std_x.transform(x_test)
std_y = StandardScaler()
y_train = std_y.fit_transform(y_train.reshape(-1,1)) #reshape(-1,1)是为了将数据转化为列向量
y_test = std_y.fit_transform(y_test.reshape(-1,1))


## 3. 使用岭回归对数据集进行预测

In [3]:
#使用岭回归进行预测
#from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
# RidgeCV可以自动选择alpha
# rd = Ridge(alpha=1.0)
rd = RidgeCV()
rd.fit(x_train,y_train)
y_rd_predict = rd.predict(x_test)
y_rd_predict = std_y.inverse_transform(y_rd_predict)


## 4. 使用Lasso回归进行预测

In [4]:
#使用Lasso回归进行预测
from sklearn.linear_model import LassoCV
#from sklearn.linear_model import Lasso
#LassoCV能自动选择alpha值
ls = LassoCV()
ls.fit(x_train,y_train)
y_ls_predict = ls.predict(x_test)
y_ls_predict = std_y.inverse_transform(y_ls_predict)


## 5. 比较两种模型预测的准确性

In [5]:
print("岭回归的MSE是： ",mean_squared_error(std_y.inverse_transform(y_test),y_rd_predict))
print("岭回归的R^2是： ",r2_score(std_y.inverse_transform(y_test),y_rd_predict))
print("Lasso回归的MSE是： ",mean_squared_error(std_y.inverse_transform(y_test),y_ls_predict))
print("Lasso回归的R^2是： ",r2_score(std_y.inverse_transform(y_test),y_ls_predict))

# 练习题
使用sklearn提供的岭回归和Lasso回归的API对糖尿病数据集的数据进行预测，并尝试将预测的结果进行分析。糖尿病的数据集使用方法如下:
```
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
```

# 参考资料
1. https://zhuanlan.zhihu.com/p/30535220
2. https://www.zhihu.com/question/297434474/answer/1001784382
3.https://blog.csdn.net/weixin_43329700/article/details/106317931?ops_request_misc=%257B%2522request%255Fid%2522%253A%2522162553626416780255258202%2522%252C%2522scm%2522%253A%252220140713.130102334..%2522%257D&request_id=162553626416780255258202&biz_id=0&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduend~default-3-106317931.first_rank_v2_pc_rank_v29&utm_term=%E6%96%B9%E5%B7%AE%E8%86%A8%E8%83%80%E5%9B%A0%E5%AD%90&spm=1018.2226.3001.4187